In [1]:
ROLE_COLS=['DevType']
TECH_COLS=[
'LanguageHaveWorkedWith',
 'MiscTechHaveWorkedWith',
 'NEWCollabToolsHaveWorkedWith',
 'PlatformHaveWorkedWith',
 'ToolsTechHaveWorkedWith',
 'WebframeHaveWorkedWith',
 'DatabaseHaveWorkedWith'
 ]

DATA_DIR='../data/processed/2_cleaned_data.pkl'
MLFLOW_TRACK_URI='../models/mlruns/'
MLFLOW_EXPERIMENT_NAME = "skills_jobs_stackoverflow"

LOG_PATH = "../models/temp/"
LOG_DATA_PKL    =  "data.pkl"
LOG_MODEL_PKL   =  "model.pkl"
LOG_METRICS_PKL =  "metrics.pkl"


In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
from pathlib import Path
import mlflow
from mlflow.tracking import MlflowClient

from sklearn.preprocessing import StandardScaler,RobustScaler
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV

from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline

from sklearn.decomposition import PCA
from sklearn.metrics import precision_score,recall_score,accuracy_score,f1_score

In [3]:
df=pd.read_pickle(DATA_DIR)
set(df.columns.droplevel(1))

{'DatabaseHaveWorkedWith',
 'DevType',
 'LanguageHaveWorkedWith',
 'MiscTechHaveWorkedWith',
 'NEWCollabToolsHaveWorkedWith',
 'PlatformHaveWorkedWith',
 'ToolsTechHaveWorkedWith',
 'WebframeHaveWorkedWith',
 'feature_clusters'}

In [4]:
df

DevType                                                \
      Data or business analyst Data scientist or machine learning specialist   
0                            0                                             0   
3                            0                                             0   
4                            0                                             0   
8                            0                                             0   
9                            0                                             1   
...                        ...                                           ...   
83434                        0                                             0   
83435                        0                                             0   
83436                        0                                             1   
83437                        0                                             0   
83438                        0                                             0   

                                                                      \
      Database administrator DevOps specialist Developer, QA or test   
0                          0                 0                     0   
3                          0                 0                     0   
4                          0                 0                     0   
8                          0                 0                     0   
9                          0                 0                     0   
...                      ...               ...                   ...   
83434                      0                 0                     0   
83435                      0                 0                     0   
83436                      0                 1                     0   
83437                      0                 0                     0   
83438                      0                 0                     0   

                                                                         \
      Developer, back-end Developer, desktop or enterprise applications   
0                       0                                             0   
3                       0                                             0   
4                       0                                             1   
8                       0                                             0   
9                       0                                             0   
...                   ...                                           ...   
83434                   1                                             0   
83435                   0                                             0   
83436                   0                                             0   
83437                   1                                             0   
83438                   1                                             0   

                                                                        \
      Developer, embedded applications or devices Developer, front-end   
0                                               0                    0   
3                                               0                    1   
4                                               0                    0   
8                                               0                    1   
9                                               0                    0   
...                                           ...                  ...   
83434                                           0                    0   
83435                                           0                    0   
83436                                           0                    0   
83437                                           0                    0   
83438                                           0                    1   

                             ... feature_clusters                       \
      Developer, full-stack  ...

In [5]:
df["DevType"].sum()

Data or business analyst                          1699
Data scientist or machine learning specialist     2591
Database administrator                            1174
DevOps specialist                                 2874
Developer, QA or test                             1113
Developer, back-end                              16977
Developer, desktop or enterprise applications     4863
Developer, embedded applications or devices       2162
Developer, front-end                              8938
Developer, full-stack                            20620
Developer, game or graphics                        919
Developer, mobile                                 4766
Engineer, data                                    1945
Engineer, site reliability                         890
Scientist                                         1191
System administrator                              1978
dtype: int64

In [6]:
samples_per_columns=1200
roles=df['DevType'].copy()
all_indices=[]
for job in roles.columns:
    # print(job)
    mask=roles[job].loc[roles[job]==1]

    if (len(mask)>samples_per_columns):
        aa=mask.sample(samples_per_columns,replace=False,random_state=0)
    if (len(mask)<samples_per_columns):
        aa=mask.sample(samples_per_columns,replace=True,random_state=0)
    all_indices.append(list(aa.index))        

all_indices=sum(all_indices,[])
new_df=df.loc[all_indices]
new_df

DevType                                                \
      Data or business analyst Data scientist or machine learning specialist   
33883                        1                                             0   
250                          1                                             1   
9101                         1                                             0   
63309                        1                                             1   
32625                        1                                             0   
...                        ...                                           ...   
24718                        0                                             0   
42768                        0                                             0   
10847                        0                                             0   
44020                        0                                             0   
81352                        0                                             0   

                                                                      \
      Database administrator DevOps specialist Developer, QA or test   
33883                      0                 0                     0   
250                        1                 0                     0   
9101                       0                 0                     0   
63309                      0                 0                     0   
32625                      0                 0                     0   
...                      ...               ...                   ...   
24718                      0                 0                     0   
42768                      0                 0                     0   
10847                      1                 0                     0   
44020                      0                 0                     0   
81352                      0                 1                     0   

                                                                         \
      Developer, back-end Developer, desktop or enterprise applications   
33883                   0                                             0   
250                     0                                             0   
9101                    0                                             0   
63309                   0                                             0   
32625                   1                                             0   
...                   ...                                           ...   
24718                   0                                             1   
42768                   0                                             0   
10847                   1                                             0   
44020                   0                                             1   
81352                   0                                             0   

                                                                        \
      Developer, embedded applications or devices Developer, front-end   
33883                                           0                    1   
250                                             0                    0   
9101                                            0                    0   
63309                                           0                    0   
32625                                           0                    0   
...                                           ...                  ...   
24718                                           0                    0   
42768                                           0                    0   
10847                                           0                    0   
44020                                           0                    0   
81352                                           0                    0   

                             ... feature_clusters                       \
      Developer, full-stack  ...

In [7]:
new_df["DevType"].sum(axis=0)

Data or business analyst                         1965
Data scientist or machine learning specialist    2313
Database administrator                           1767
DevOps specialist                                2513
Developer, QA or test                            1490
Developer, back-end                              5855
Developer, desktop or enterprise applications    2736
Developer, embedded applications or devices      1836
Developer, front-end                             2638
Developer, full-stack                            5730
Developer, game or graphics                      1448
Developer, mobile                                2134
Engineer, data                                   2020
Engineer, site reliability                       1603
Scientist                                        1652
System administrator                             2307
dtype: int64

In [8]:
Path(MLFLOW_TRACK_URI).mkdir(parents=True,exist_ok=True)
Path(LOG_PATH).mkdir(parents=True,exist_ok=True)

In [9]:
X_train,X_test,y_train,y_test=train_test_split(new_df.drop('DevType',axis=1),new_df['DevType'],test_size=.3,random_state=52)

C:\Users\Mohamed Mosaad\AppData\Local\Temp\ipykernel_10804\2140877071.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  X_train,X_test,y_train,y_test=train_test_split(new_df.drop('DevType',axis=1),new_df['DevType'],test_size=.3,random_state=52)


In [10]:
print(X_train.shape)
print(y_train.shape)

(13440, 145)
(13440, 16)


Models

In [11]:
# Initialize client and experiment
mlflow.set_tracking_uri(MLFLOW_TRACK_URI)
client = MlflowClient()
mlflow.get_experiment_by_name(MLFLOW_EXPERIMENT_NAME)
exp = client.get_experiment_by_name(MLFLOW_EXPERIMENT_NAME)

Vanilla random forest


In [12]:
vanilla_rf=make_pipeline(StandardScaler(),
                         PCA(n_components=.95)
                         ,RandomForestClassifier(random_state=0))
vanilla_rf.fit(X_train,y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pca', PCA(n_components=0.95)),
                ('randomforestclassifier',
                 RandomForestClassifier(random_state=0))])

In [13]:
y_train_predicted_1=vanilla_rf.predict(X_train)
y_test_predicted_1=vanilla_rf.predict(X_test)

In [14]:
y_train_predicted_1=pd.DataFrame(y_train_predicted_1,columns=y_train.columns)
y_test_predicted_1=pd.DataFrame(y_test_predicted_1,columns=y_train.columns)
y_test_predicted_1

,Data or business analyst,Data scientist or machine learning specialist,Database administrator,DevOps specialist,"Developer, QA or test","Developer, back-end","Developer, desktop or enterprise applications","Developer, embedded applications or devices","Developer, front-end","Developer, full-stack","Developer, game or graphics","Developer, mobile","Engineer, data","Engineer, site reliability",Scientist,System administrator
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5755,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5756,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
5757,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1
5758,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [15]:
def metrices(y,y_predicted,metrics=[accuracy_score,precision_score,recall_score,f1_score]):
    scores={}
    for col in y.columns:
        scores[col]=[metric(y[col],y_predicted[col]) for metric in metrics]
    return(pd.DataFrame(scores,index=['accuracy_score','precision_score','recall_score','f1_score']).T) 

In [16]:
scores={}
for col in y_train.columns:
    scores[col]=[metric(y_train[col],y_train_predicted_1[col]) for metric in [accuracy_score,precision_score,recall_score,f1_score]]
scores_train_1=pd.DataFrame(scores,index=['accuracy_score','precision_score','recall_score','f1_score']).T
scores_train_1

,accuracy_score,precision_score,recall_score,f1_score
Data or business analyst,0.999777,1.000000,0.997859,0.998928
Data scientist or machine learning specialist,0.999851,1.000000,0.998766,0.999383
Database administrator,0.999777,1.000000,0.997588,0.998793
DevOps specialist,0.999777,0.998881,0.999440,0.999161
"Developer, QA or test",0.999628,0.999042,0.996180,0.997609
"Developer, back-end",0.999554,0.999272,0.999272,0.999272
"Developer, desktop or enterprise applications",0.999033,0.997936,0.995368,0.996650
"Developer, embedded applications or devices",0.999628,0.998439,0.997660,0.998049
"Developer, front-end",0.999479,0.998889,0.997227,0.998057
"Developer, full-stack",0.999777,0.999497,0.999749,0.999623


In [17]:
metrices(y_train,y_train_predicted_1)

,accuracy_score,precision_score,recall_score,f1_score
Data or business analyst,0.999777,1.000000,0.997859,0.998928
Data scientist or machine learning specialist,0.999851,1.000000,0.998766,0.999383
Database administrator,0.999777,1.000000,0.997588,0.998793
DevOps specialist,0.999777,0.998881,0.999440,0.999161
"Developer, QA or test",0.999628,0.999042,0.996180,0.997609
"Developer, back-end",0.999554,0.999272,0.999272,0.999272
"Developer, desktop or enterprise applications",0.999033,0.997936,0.995368,0.996650
"Developer, embedded applications or devices",0.999628,0.998439,0.997660,0.998049
"Developer, front-end",0.999479,0.998889,0.997227,0.998057
"Developer, full-stack",0.999777,0.999497,0.999749,0.999623


In [18]:
scores_test_1=metrices(y_test,y_test_predicted_1)
scores_test_1

,accuracy_score,precision_score,recall_score,f1_score
Data or business analyst,0.945486,0.931034,0.478723,0.632319
Data scientist or machine learning specialist,0.942014,0.903153,0.579480,0.705986
Database administrator,0.969271,0.994286,0.665392,0.797251
DevOps specialist,0.932292,0.969274,0.477961,0.640221
"Developer, QA or test",0.970486,1.000000,0.616253,0.762570
"Developer, back-end",0.808160,0.874702,0.422966,0.570206
"Developer, desktop or enterprise applications",0.913889,0.940653,0.399748,0.561062
"Developer, embedded applications or devices",0.943403,0.895833,0.465704,0.612827
"Developer, front-end",0.898264,0.910891,0.330539,0.485062
"Developer, full-stack",0.833160,0.818841,0.580148,0.679132


In [19]:
print(scores_test_1.mean())
mean_test_scores=scores_test_1.mean()

accuracy_score     0.931836
precision_score    0.945206
recall_score       0.537877
f1_score           0.679435
dtype: float64


Log

In [20]:
## Log
# Data details
data_details = {"data_path": DATA_DIR,
                "training_indices": X_train.index.tolist(),
                "test_indices": X_test.index.tolist(),
                "features_names": X_train.columns.droplevel(0).tolist(),
                "targets_names": y_train.columns.tolist()}

with open(os.path.join(LOG_PATH, LOG_DATA_PKL), "wb") as output_file:
    pickle.dump(data_details, output_file)

In [21]:
# Model
model = {"model_description": "Random Forest: with PCA - Basic",
         "model_details": str(vanilla_rf),
         "model_object": vanilla_rf}

with open(os.path.join(LOG_PATH, LOG_MODEL_PKL), "wb") as output_file:
    pickle.dump(model, output_file)

In [22]:
# Performance details
classes_metrics = {"train_scores": scores_train_1,
                   "test_scores": scores_test_1}

with open(os.path.join(LOG_PATH, LOG_METRICS_PKL), "wb") as output_file:
    pickle.dump(classes_metrics, output_file)

In [23]:
with mlflow.start_run(experiment_id=exp.experiment_id,
                      run_name=model["model_description"]):
    # Log pickles
    mlflow.log_artifacts(LOG_PATH)

    # Track metrics
    for metric, score in mean_test_scores.items():
        mlflow.log_metric(metric, score)

Parameter Tuning

In [24]:
# param_grid={
#     'scaler':[StandardScaler(),RobustScaler()],
#     'pca__n_components':np.arange(.6,1,.05),
#     'rf__max_features':[12,14,16,18],
#     'rf__max_depth':[3,5]

# }

# rf=make_pipeline([
#                   ('scaler',StandardScaler()),
#                   ('pca',PCA()),
#                   ('rf',RandomForestClassifier(n_estimators=300))
#                   ])
# grid=GridSearchCV(rf,param_grid=param_grid)
from sklearn.pipeline import Pipeline
# param_grid = {
#     'scaler': [StandardScaler(), RobustScaler()],
#     'pca__n_components': np.concatenate((np.arange(0.6, 1, 0.05), [5, 10, 15])),  # Add specific component values
#     'rf__max_features': ['auto', 'sqrt', 8, 12, 16],  # Test smaller values
#     'rf__max_depth': [3, 5, 10, 15, 20, None]        # Test deeper trees
# }
param_grid = {
    # 'scaler': [StandardScaler(), RobustScaler()],  # Keep two scaling options
    'pca__n_components': [0.8, 0.9],  # Use fewer PCA components, focus on high variance retention
    'rf__max_features': ['auto', 12],  # Reduce options for max_features
    'rf__max_depth': [10, 20]     # Test shallower trees, reduce the range
}

rf = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('pca', PCA()),
    ('rf', RandomForestClassifier(n_jobs=8
        ,n_estimators=200))
])

grid = GridSearchCV(rf, param_grid=param_grid,scoring='average_precision',cv=2)


In [25]:
grid

GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('pca', PCA()),
                                       ('rf',
                                        RandomForestClassifier(n_estimators=200,
                                                               n_jobs=8))]),
             param_grid={'pca__n_components': [0.8, 0.9],
                         'rf__max_depth': [10, 20],
                         'rf__max_features': ['auto', 12]},
             scoring='average_precision')

In [26]:
grid.fit(X_train.values,y_train.values)

c:\Users\Mohamed Mosaad\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
8 fits failed out of a total of 16.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
8 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Mohamed Mosaad\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Mohamed Mosaad\anaconda3\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Mohamed Mosaad\anaconda3\Lib\site-packages\sklearn\pipeline.py", line 473, in

GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('pca', PCA()),
                                       ('rf',
                                        RandomForestClassifier(n_estimators=200,
                                                               n_jobs=8))]),
             param_grid={'pca__n_components': [0.8, 0.9],
                         'rf__max_depth': [10, 20],
                         'rf__max_features': ['auto', 12]},
             scoring='average_precision')

In [27]:
y_train_predicted=grid.predict(X_train)
y_test_predicted=grid.predict(X_test)

In [28]:
y_train_predicted=pd.DataFrame(y_train_predicted,columns=y_train.columns)
y_test_predicted=pd.DataFrame(y_test_predicted,columns=y_train.columns)


In [29]:
scores_test=metrices(y_test,y_test_predicted)
scores_test

,accuracy_score,precision_score,recall_score,f1_score
Data or business analyst,0.945139,0.927586,0.476950,0.629977
Data scientist or machine learning specialist,0.944792,0.897872,0.609827,0.726334
Database administrator,0.967708,0.994135,0.648184,0.784722
DevOps specialist,0.930903,0.963277,0.469697,0.631481
"Developer, QA or test",0.967882,1.000000,0.582393,0.736091
"Developer, back-end",0.810069,0.871944,0.432198,0.577932
"Developer, desktop or enterprise applications",0.914757,0.928977,0.412358,0.571179
"Developer, embedded applications or devices",0.944965,0.885993,0.490975,0.631823
"Developer, front-end",0.900174,0.906250,0.347305,0.502165
"Developer, full-stack",0.842014,0.807889,0.630918,0.708520


In [30]:
scores_test_mean=scores_test.mean()

In [31]:
mlflow.get_tracking_uri()

'../models/mlruns/'

In [32]:
data_details = {"data_path": DATA_DIR,
                "training_indices": X_train.index.tolist(),
                "test_indices": X_test.index.tolist(),
                "features_names": X_train.columns.droplevel(0).tolist(),
                "targets_names": y_train.columns.tolist()}

with open(os.path.join(LOG_PATH, LOG_DATA_PKL), "wb") as output_file:
    pickle.dump(data_details, output_file)

In [33]:
model = {"model_description": "Random Forest: with PCA - Grid search",
         "model_details": str(grid),
         "model_object": grid}

with open(os.path.join(LOG_PATH, LOG_MODEL_PKL), "wb") as output_file:
    pickle.dump(model, output_file)

In [34]:
# Performance details
classes_metrics = {"train_scores": scores_train_1,
                   "test_scores": scores_test}

with open(os.path.join(LOG_PATH, LOG_METRICS_PKL), "wb") as output_file:
    pickle.dump(classes_metrics, output_file)

In [35]:
with mlflow.start_run(experiment_id=exp.experiment_id, 
                      run_name=model["model_description"]):
    # Log pickles
    mlflow.log_artifacts(LOG_PATH)  
    
    # Track metrics 
    for metric, score in scores_test_mean.items():
        mlflow.log_metric(metric, score) 